In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import pickle
import normalize_data
import random
import train

pd.set_option('display.max_columns', 999)
pd.set_option('display.precision', 10)

In [2]:
def get_ref_X_y(df):
    ref_cols = [c for c in df.columns if c.startswith('__')]
    X_cols = [c for c in df.columns if c.startswith('X_')]
    y_cols = [c for c in df.columns if c.startswith('y_')]
    return (df[ref_cols], df[X_cols], df[y_cols])

In [3]:
raw_data = {} # loads raw data and stores as a dict cache

def dataset_key(dataset='', validation=False):
    return dataset+('test' if validation else 'train')


def load_data(raw, dataset='', validation=False):
    '''
    Return dataframe matching data set and validation. Dictionary input will be updated.

    Parameters
    ----------
    raw : dict
        dictionary which caches the dataframes and will be updated accordingly

    dataset : str
        which dataset to use? valid input includes: empty str for full set, sample_, and secret_

    validation : bool
        load validation set? if true then use _test, otherwise use _train.  Note secret_ doesn't have _train
    '''
    key = dataset+('test' if validation else 'train')
    if key not in raw:
        print(f"Loading data to cache for: {key}")
        raw[key] = pd.read_pickle(f'{os.environ["GP_HIST_PATH"]}/../t4_data/{key}.pkl')
    return raw[key]

In [4]:
configurations = {
    'dataset' : '', # '', 'sample_', 'secret_'
    'model_identifier' : "big_three_0",
    'model_path' : f"{os.environ['GP_HIST_PATH']}/../t4_models",
    'device' : 'cpu',
    'random_seed' : 0,
    'lr' : 1e-4,
    'momentum' : 0.97, #SGD
    'weight_decay' : 1e-7, #Adam
    'max_epochs' : 500,
    'do_validate' : True,
    'model_definition' : [
        ('layer', 200), ('relu1', None),
        ('layer', 200), ('relu1', None), ('drop', 0.3),
        ('layer', 200), ('relu1', None), ('drop', 0.3),
        ('layer', 200), ('relu1', None),
        ('layer', 200), ('relu1', None), ('drop', 0.3),
        ('layer', 200), ('relu1', None), ('drop', 0.3),
        ('layer', 200), ('relu1', None),
        ('layer', 200), ('relu1', None),
        ('layer', 200), ('relu1', None), ('drop', 0.3),
        ('layer', 200), ('relu1', None), ('drop', 0.3),
        ('layer', 200), ('relu1', None),
        ('layer', 200), ('relu1', None),
        ('layer', 200), ('relu1', None),
        ('layer', 200), ('relu1', None), ('drop', 0.3),
        ('layer', 200), ('relu1', None), ('drop', 0.3),
        ('layer', 200), ('relu1', None),
        ('layer', 200), ('relu1', None),
        ('layer', 200), ('relu1', None),
        ('layer', 200), ('relu1', None), ('drop', 0.3),
        ('layer', 200), ('relu1', None), ('drop', 0.3),
        ('layer', 200), ('relu1', None),
        ('layer', 200), ('relu1', None),
        ('layer', 200), ('relu1', None),
        ('layer', 200),
    ],
    'train_params' : {
        'batch_size': 50000,
        'shuffle': True,
        'num_workers': 3,
        'pin_memory': True,
    },
    'test_params' : {
        'batch_size': 200000,
        'num_workers': 3,
        'pin_memory': True,
    },
}

In [ ]:
%%time

train_df = normalize_data.normalize_all_columns(load_data(raw_data,dataset=configurations['dataset'],validation=False))
test_df = normalize_data.normalize_all_columns(load_data(raw_data,dataset=configurations['dataset'],validation=True))

train_df = train_df.dropna()
test_df = test_df.dropna()
# train_df = train_df.sample(100000, random_state=42)
# test_df = test_df.sample(10000, random_state=42)

ref_train, X_train, y_train = get_ref_X_y(train_df)
ref_test, X_test, y_test = get_ref_X_y(test_df)

Loading data to cache for: train


In [ ]:
y_cols = ['y_BSTAR', 'y_INCLINATION', 'y_RA_OF_ASC_NODE', 'y_ECCENTRICITY', 'y_ARG_OF_PERICENTER', 'y_MEAN_MOTION']
y_cols = ['y_INCLINATION', 'y_ECCENTRICITY', 'y_MEAN_MOTION']

y_train = y_train[y_cols]
y_test = y_test[y_cols]

# y_train = np.sign(y_train) * np.log(1+np.abs(y_train))
# y_test = np.sign(y_test) * np.log(1+np.abs(y_test))

In [ ]:
model, mean_losses = train.train_model(X_train, y_train, X_test, y_test, configurations, force_train=False)

In [ ]:
model, _, _, mean_losses, _ = train.load_model_with_config(configurations)

tl, vl = zip(*mean_losses)

fig,ax = plt.subplots()
ax.plot(tl, label="Training Loss")
ax.plot(vl, label="Validation Loss")

fig.legend()
plt.show()

In [ ]:
trained_model = model

In [ ]:
# y_train_pred = train.predict(trained_model, X_train, y_train, device="cpu") # get predictions for each train
# y_train_pred_df = pd.DataFrame(y_train_pred, columns=y_train.columns)  # put results into a dataframe
y_test_pred = train.predict(trained_model, X_test, y_test, device="cpu") # get predictions for each train
y_test_pred_df = pd.DataFrame(y_test_pred, columns=y_test.columns)  # put results into a dataframe

In [ ]:
print(f'    Test set MAE (L1) loss: {mean_absolute_error(y_test, y_test_pred_df)}')
print(f'    Test set MSE (L2) loss: {mean_squared_error(y_test, y_test_pred_df)}')

# random.seed(342798)
sample = random.sample(list(y_test_pred_df.index), 10)
# sample = [0,1]

print("Test - Ground Truth (normalized):")
display(y_test.loc[sample])
print("Test - Prediction (normalized):")
display(y_test_pred_df.loc[sample])
print("Ground Truth Diffs from X_:")
display(y_test.loc[sample] - X_test.loc[sample][['X_INCLINATION_1','X_ECCENTRICITY_1','X_MEAN_MOTION_1']].values)
print("Pred Diffs from X_:")
display(y_test_pred_df.loc[sample] - X_test.loc[sample][['X_INCLINATION_1','X_ECCENTRICITY_1','X_MEAN_MOTION_1']].values)


In [ ]:
# print(f'    Train set MAE (L1) loss: {mean_absolute_error(y_train, y_train_pred_df)}')
# print(f'    Train set MSE (L2) loss: {mean_squared_error(y_train, y_train_pred_df)}')

# random.seed(0)
# sample = random.sample(list(y_train_pred_df.index), 10)

# print("Train - Ground Truth (normalized):")
# display(y_train.loc[sample])
# # print("Train - Ground Truth (non-normalized):")
# # display(normalize_data.normalize_all_columns(y_train.iloc[:,3:].loc[sample].copy(), reverse=True))  # see ground truths
# print("Train - Prediction (normalized):")
# display(y_train_pred_df.loc[sample])
# # print("Train - Prediction (non-normalized):")
# # display(normalize_data.normalize_all_columns(y_train_pred_df.loc[sample].copy(), reverse=True))  # See predictions

In [ ]:
# errors = y_train.loc[sample].iloc[:,3:] - y_train_pred_df.loc[sample]

# display(errors)

# display(errors.std())

# display(y_train.loc[sample].iloc[:,3:])
# display(y_train_pred_df.loc[sample])

In [ ]:
# def row_to_compare(X, y, y_pred, row):
#     epoch = X.iloc[row].EPOCH_y
#     X0 = clean_data.normalize_all_columns(X_train.iloc[row].copy(), reverse=True)
#     y0 = clean_data.normalize_all_columns(y_train.iloc[row].copy(), reverse=True)
#     y1 = clean_data.normalize_all_columns(y_train_pred_df.iloc[row].copy(), reverse=True)

#     # Ground truth
#     y0_xyz = clean_data.get_satellite_xyz(bst=0.0001,
#                                           ecc=y0.ECCENTRICITY,
#                                           aop=y0.ARG_OF_PERICENTER,
#                                           inc=y0.INCLINATION,
#                                           mea=y0.MEAN_ANOMALY,
#                                           mem=y0.MEAN_MOTION,
#                                           raa=y0.RA_OF_ASC_NODE,
#                                           epoch=epoch,)
#     # Prediction
#     y1_xyz = clean_data.get_satellite_xyz(bst=0.0001,
#                                           ecc=y1.ECCENTRICITY,
#                                           aop=y1.ARG_OF_PERICENTER,
#                                           inc=y1.INCLINATION,
#                                           mea=y1.MEAN_ANOMALY,
#                                           mem=y1.MEAN_MOTION,
#                                           raa=y1.RA_OF_ASC_NODE,
#                                           epoch=epoch,)
#     # Propigation
#     y2_xyz = clean_data.get_satellite_xyz(bst=X0.BSTAR,
#                                           ecc=X0.ECCENTRICITY,
#                                           aop=X0.ARG_OF_PERICENTER,
#                                           inc=X0.INCLINATION,
#                                           mea=X0.MEAN_ANOMALY,
#                                           mem=X0.MEAN_MOTION,
#                                           raa=X0.RA_OF_ASC_NODE,
#                                           epoch=epoch,)
#     print(f'Ground Truth: {y0_xyz}')
#     print(f'Predicted: {y1_xyz}')
#     print(f'Propigation: {y2_xyz}')
    
#     print (f'Prediction Error: {sum((y1_xyz-y0_xyz)**2)**0.5} km')
#     print (f'Propigation Error: {sum((y2_xyz-y0_xyz)**2)**0.5} km')
    

# for row in range(2):
#     print (f'Row {row}:')
#     row_to_compare(X_train, y_train, y_train_pred_df, row)
#     print('')